In [4]:
from tools import open_json


In [2]:
data = open_json('2023_season.json')
print(len(data))

285


In [6]:
game1 = data['20230911_BUF@NYJ']
print(game1.keys())

dict_keys(['statusCode', 'body'])


In [10]:
print(game1['body']['playerStats'].keys())
print(game1['body']['playerStats']['2577466'])

dict_keys(['4039007', '14053', '4035004', '3043136', '3120464', '4240090', '3125232', '4037584', '3915486', '11260', '2581273', '3122168', '3895791', '3125114', '3916577', '2976549', '3917660', '3930086', '4033748', '2977679', '3046287', '4569618', '4240456', '4240896', '4240532', '4427366', '3051324', '3117258', '4240657', '4428696', '3042717', '16910', '3895798', '16759', '4411193', '3121003', '4361767', '2582132', '13493', '2983155', '3128390', '3929956', '4240585', '4427250', '4567962', '3858271', '4243537', '2577466', '4039375', '3929631', '16720', '4379399', '2970625', '4037521', '4039303', '2512477', '15960', '4361259', '2577078', '3918298', '3139387', '4361331', '4259646', '4030899', '2976249', '4039505', '4361372', '15928', '3057524', '3110565', '4259166', '3925347', '15920', '4040982', '14993', '4568624', '2577637', '4035232', '4385690', '3116593', '3917232', '2976212', '2574576', '4428681', '4427278', '4373678', '8439', '2577367', '2514122', '4379778', '4362506', '15979', '4

In [7]:
#for all games, we want log the stats of each participating player.

all_players = {}


for game_id in data.keys():
    player_stats = data[game_id]['body']['playerStats']
    for player_id, player_stat in player_stats.items():
        if player_id not in all_players: all_players[player_id] = []
        all_players[player_id].append(player_stat)


In [8]:

print(len(all_players))


2145


In [28]:
#let's get the total tackles across a whole season of a specific player

taylor = all_players['4039007']
season_tackles = [taylor[i]['Defense']['totalTackles'] for i in range(len(taylor)) if 'Defense' in taylor[i]]
print(season_tackles)

['5', '1', '1', '6', '1', '1', '3', '7', '3', '3', '8', '9', '1', '1']


In [9]:
all_players_clean = {}

# let's try separating players by position
for player_id, player_stat in all_players.items():
    if len(player_stat) > 6:
        all_players_clean[player_id] = player_stat

In [49]:
#print(len(all_players_clean))
import numpy as np

#get average size of player stats
#print(np.mean(np.array([len(player_stat) for player_stat in all_players_clean.values()])))


player_names = [player[0]['longName'] for player in all_players_clean.values()]

In [5]:
all_players_clean = open_json('all_players_clean_2023.json')

In [4]:
print(len(player_names))
print(len(set(player_names)))
print(len(all_players_clean.items()))

NameError: name 'player_names' is not defined

In [6]:
import random
rand_player = random.randint(0,len(all_players_clean))
rand_id = list(all_players_clean.keys())[rand_player]
print(all_players_clean[rand_id][0])
print(all_players_clean[rand_id][0]['longName'])
print(all_players_clean[rand_id][0]['playerID'])

{'gameID': '20230911_BUF@NYJ', 'teamID': '25', 'snapCounts': {'offSnapPct': '0.00', 'defSnap': '69', 'stSnap': '4', 'stSnapPct': '0.15', 'offSnap': '0', 'defSnapPct': '1.00'}, 'team': 'NYJ', 'teamAbv': 'NYJ', 'Defense': {'totalTackles': '7', 'defTD': '0', 'forcedFumbles': '0', 'soloTackles': '5', 'tfl': '0', 'qbHits': '0', 'defensiveInterceptions': '0', 'sacks': '0', 'passDeflections': '0'}, 'playerID': '16720', 'longName': 'C.J. Mosley'}
C.J. Mosley
16720


In [7]:
from player import Player

play = Player('f','a')
play.total_forcedFumbles.append('a')


In [14]:
from player import Player

def record_player_season_stats(player_id: str, all_players_dict: dict, season: int, existing_player: Player = None) -> Player:
    assert player_id in all_players_dict.keys() and len(list(all_players_dict[player_id])) >= 8
    assert 1970 <= season <= 2024
    player_name = all_players_dict[player_id][0]['longName']
    print(player_name)
    
    player = Player(player_id, player_name) if not existing_player else existing_player
    
    entire_season = all_players_dict[player_id]

    for game in entire_season:
        player.total_games_recorded += 1
        if 'snapCounts' in game:
            sc = game['snapCounts']
            for stat_name in ['offSnapPct','offSnap','defSnapPct','defSnap','stSnapPct','stSnap']:
                if stat_name in sc:
                    player.add_stat(season, stat_name, float(sc[stat_name]))

        if 'Rushing' in game:
            rs = game['Rushing']
            for stat_name in ['rushAvg','rushYds','carries','rushTD']:
                if stat_name in rs:
                    player.add_stat(season, stat_name, float(rs[stat_name]))

        if 'Receiving' in game:
            rc = game['Receiving']
            for stat_name in ['receptions','recYds','recAvg','targets']:
                if stat_name in rc:
                    player.add_stat(season, stat_name, float(rc[stat_name]))

        if 'Passing' in game:
            ps = game['Passing']
            for stat_name in ['qbr','rtg','passAttempts','passAvg','passTD','passYds','int','passCompletions']:
                if stat_name in ps:
                    player.add_stat(season, stat_name, float(ps[stat_name]))

        if 'Defense' in game:
            df = game['Defense']
            for stat_name in ['totalTackles','soloTackles','forcedFumbles','sacks','defensiveInterceptions','passDeflections']:
                if stat_name in df:
                    player.add_stat(season, stat_name, float(df[stat_name]))

    return player


pid = '4430807'

p = record_player_season_stats(pid, all_players_clean)

Bijan Robinson


In [10]:
from tools import save_json

save_json('all_players_clean_2023.json',all_players_clean)

Data has been saved to all_players_clean_2023.json


In [18]:
import json
print(json.dumps(p.__dict__,indent=2))

{
  "total_games_recorded": 17,
  "player_id": "4430807",
  "full_name": "Bijan Robinson",
  "total_offensive_snapPct": [
    0.75,
    0.61,
    0.75,
    0.76,
    0.74,
    0.77,
    0.81,
    0.61,
    0.72,
    0.76,
    0.17,
    0.78,
    0.63,
    0.75,
    0.63,
    0.58,
    0.74
  ],
  "total_offensive_snaps": [
    51.0,
    47.0,
    48.0,
    44.0,
    50.0,
    64.0,
    54.0,
    42.0,
    59.0,
    45.0,
    11.0,
    56.0,
    33.0,
    53.0,
    39.0,
    30.0,
    45.0
  ],
  "total_defensive_snapPct": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "total_defensive_snaps": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "total_st_snapPct": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    